In [113]:
import os
import time

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

from credentials import set_credentials

In [25]:
pd.options.display.max_columns = None

# Spotify Recommendation System Walkthrough

## Spotify Access
How to access Spotify through their developer API

In [26]:
set_credentials()

In [27]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('CLIENT_ID'), client_secret=os.getenv('CLIENT_SECRET'))
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Extract Tracks from a Playlist

Spotify makes use of a URI (Universal resource locator) in order to identify playlsits, artists, tracks, ect. For example from the link: `https://open.spotify.com/playlist/2QJwkjEtDXDbtecQ5ypVm3?si=c437b89583c74615` to one of my favourite playlists, only the `2QJwkjEtDXDbtecQ5ypVm3` is the URI.

In [28]:
playlist_link = "https://open.spotify.com/playlist/2QJwkjEtDXDbtecQ5ypVm3?si=c437b89583c74615"  # Motivate
# playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"  # Tutorial

In [29]:
playlist_uri = playlist_link.split('/')[-1].split('?')[0]
print(playlist_uri)

2QJwkjEtDXDbtecQ5ypVm3


In [30]:
mot_playlist = sp.playlist_tracks(playlist_uri, limit=100, offset=0)

In [31]:
print(mot_playlist)

{'href': 'https://api.spotify.com/v1/playlists/2QJwkjEtDXDbtecQ5ypVm3/tracks?offset=0&limit=100&additional_types=track', 'items': [{'added_at': '2020-11-14T05:40:14Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/fn662s8jr8skpfl5kn81itq7c'}, 'href': 'https://api.spotify.com/v1/users/fn662s8jr8skpfl5kn81itq7c', 'id': 'fn662s8jr8skpfl5kn81itq7c', 'type': 'user', 'uri': 'spotify:user:fn662s8jr8skpfl5kn81itq7c'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3NPpFNZtSTHheNBaWC82rB'}, 'href': 'https://api.spotify.com/v1/artists/3NPpFNZtSTHheNBaWC82rB', 'id': '3NPpFNZtSTHheNBaWC82rB', 'name': 'X Ambassadors', 'type': 'artist', 'uri': 'spotify:artist:3NPpFNZtSTHheNBaWC82rB'}], 'available_markets': ['AD', 'AE', 'AG', 'AR', 'AT', 'AU', 'BB', 'BE', 'BG', 'BH', 'BO', 'BR', 'BS', 'BZ', 'CA', 'CH', 'CL', 'CO', 'CR', 'CW', 'CY', 'CZ', 'DE', 'DK', 'DM', 'DO', '

Display the total number of songs in the playlist

In [32]:
print(mot_playlist['total'])

301


Display author of playlist (spotify link)

In [37]:
items = mot_playlist['items']

In [38]:
print(items)

[{'added_at': '2020-11-14T05:40:14Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/fn662s8jr8skpfl5kn81itq7c'}, 'href': 'https://api.spotify.com/v1/users/fn662s8jr8skpfl5kn81itq7c', 'id': 'fn662s8jr8skpfl5kn81itq7c', 'type': 'user', 'uri': 'spotify:user:fn662s8jr8skpfl5kn81itq7c'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3NPpFNZtSTHheNBaWC82rB'}, 'href': 'https://api.spotify.com/v1/artists/3NPpFNZtSTHheNBaWC82rB', 'id': '3NPpFNZtSTHheNBaWC82rB', 'name': 'X Ambassadors', 'type': 'artist', 'uri': 'spotify:artist:3NPpFNZtSTHheNBaWC82rB'}], 'available_markets': ['AD', 'AE', 'AG', 'AR', 'AT', 'AU', 'BB', 'BE', 'BG', 'BH', 'BO', 'BR', 'BS', 'BZ', 'CA', 'CH', 'CL', 'CO', 'CR', 'CW', 'CY', 'CZ', 'DE', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GD', 'GR', 'GT', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS',

In [39]:
len(items)

100

Record track information

In [40]:
track_info = {
    'uris': [],
    'names': [],
    'artist_names': [],
    'artist_pop': [],
    'artist_genres': [],
    'albums': [],
    'track_pop': [],
    'danceability': [],
    'energy': [],
    'keys': [],
    'loudness': [],
    'modes': [],
    'speechiness': [],
    'acousticness': [],
    'instrumentalness': [],
    'liveness': [],
    'valences': [],
    'tempos': [],
    'types': [],
    'ids': [],
    'track_hrefs': [],
    'analysis_urls': [],
    'durations_ms': [],
    'time_signatures': []
}

In [41]:
for item in items:
    track_uri = item['track']['uri']
    track_info['uris'].append(track_uri)  # Retrieve track uri
    track_info['names'].append(item['track']['name'])  # Retrieve track name

    artist_uri = item['track']['artists'][0]['uri']  # Find artist uri
    artist_info = sp.artist(artist_uri)  # Extract information on artist

    track_info['artist_names'].append(item['track']['artists'][0]['name'])  # Access artist name
    track_info['artist_pop'].append(artist_info['popularity'])  # Access 
    track_info['artist_genres'].append(artist_info['genres'])  # Access artist genres

    track_info['albums'].append(item['track']['album']['name'])  # Access album names
    track_info['track_pop'].append(item['track']['popularity'])  # Access track popularity

    track_features = sp.audio_features(track_uri)[0]  # Access track features
    track_info['danceability'].append(track_features['danceability'])
    track_info['energy'].append(track_features['energy'])
    track_info['keys'].append(track_features['key'])
    track_info['loudness'].append(track_features['loudness'])
    track_info['modes'].append(track_features['mode'])
    track_info['speechiness'].append(track_features['speechiness'])
    track_info['acousticness'].append(track_features['acousticness'])
    track_info['instrumentalness'].append(track_features['instrumentalness'])
    track_info['liveness'].append(track_features['liveness'])
    track_info['valences'].append(track_features['valence'])
    track_info['tempos'].append(track_features['tempo'])
    track_info['types'].append(track_features['type'])
    track_info['ids'].append(track_features['id'])
    track_info['track_hrefs'].append(track_features['track_href'])
    track_info['analysis_urls'].append(track_features['analysis_url'])
    track_info['durations_ms'].append(track_features['duration_ms'])
    track_info['time_signatures'].append(track_features['time_signature'])

In [42]:
print(track_info)

{'uris': ['spotify:track:3V9cM3nCH2G66afoDi0snu', 'spotify:track:0fYVliAYKHuPmECRs1pbRf', 'spotify:track:2NldmECxe8jE4TmLmu5Yao', 'spotify:track:3xgK660fsZH7ZDcOMfIdfB', 'spotify:track:7o2CTH4ctstm8TNelqjb51', 'spotify:track:2HrsT9eflRVvdu3ZrorjQ6', 'spotify:track:04aAxqtGp5pv12UXAg4pkq', 'spotify:track:6bYjclvTLjcmZZC5GhmSFQ', 'spotify:track:6jXJmMCmEXC8SmZTK0KId6', 'spotify:track:3qiyyUfYe7CRYLucrPmulD', 'spotify:track:6x4tKaOzfNJpEJHySoiJcs', 'spotify:track:7bEDDsy2LFC0KSqhZp5nPE', 'spotify:track:1ejq4bPmYNhITVsCGNnVv2', 'spotify:track:4Ps2HEc6cRSy3fKPYfGAQ3', 'spotify:track:2lEXaTrSES6eZyypyvo9yR', 'spotify:track:3qgPpwtuRu5oP8EtFSj8HE', 'spotify:track:1T02MleMfYPohqKcd8uY6J', 'spotify:track:1NWfdpuChB7HcctVvDvGIy', 'spotify:track:6EjfjKpSk2eb8aAcX2D08Y', 'spotify:track:6JsiDFQRx7GHcuf2UroSYB', 'spotify:track:2ouTPeXZDbhcm2R2zx5LBG', 'spotify:track:63AXfaJ12wKlnXe9QAvzbl', 'spotify:track:7zzoxJbgjme3366mOp5UnH', 'spotify:track:40AHsBVOfw3H9i1jIrJEbb', 'spotify:track:10Nmj3JCNoMeBQ8

### DataFrame the Information

In [43]:
df = pd.DataFrame.from_dict(track_info)

In [44]:
df.shape

(100, 24)

In [45]:
df.head()

,uris,names,artist_names,artist_pop,artist_genres,albums,track_pop,danceability,energy,keys,loudness,modes,speechiness,acousticness,instrumentalness,liveness,valences,tempos,types,ids,track_hrefs,analysis_urls,durations_ms,time_signatures
0,spotify:track:3V9cM3nCH2G66afoDi0snu,BOOM,X Ambassadors,70,"[modern alternative rock, modern rock, stomp pop]",ORION,61,0.732,0.594,11,-5.911,0,0.0693,0.008050,0.030100,0.0802,0.676,107.996,audio_features,3V9cM3nCH2G66afoDi0snu,https://api.spotify.com/v1/tracks/3V9cM3nCH2G6...,https://api.spotify.com/v1/audio-analysis/3V9c...,164587,4
1,spotify:track:0fYVliAYKHuPmECRs1pbRf,Renegades,X Ambassadors,70,"[modern alternative rock, modern rock, stomp pop]",VHS,78,0.526,0.862,2,-6.003,1,0.0905,0.014400,0.059700,0.2290,0.528,90.052,audio_features,0fYVliAYKHuPmECRs1pbRf,https://api.spotify.com/v1/tracks/0fYVliAYKHuP...,https://api.spotify.com/v1/audio-analysis/0fYV...,195200,4
2,spotify:track:2NldmECxe8jE4TmLmu5Yao,Hey Child,Korbee,17,[],Hey Child,33,0.649,0.796,9,-3.890,1,0.0328,0.002670,0.000157,0.3620,0.653,140.035,audio_features,2NldmECxe8jE4TmLmu5Yao,https://api.spotify.com/v1/tracks/2NldmECxe8jE...,https://api.spotify.com/v1/audio-analysis/2Nld...,241267,4
3,spotify:track:3xgK660fsZH7ZDcOMfIdfB,Jungle,X Ambassadors,70,"[modern alternative rock, modern rock, stomp pop]",VHS,60,0.389,0.748,2,-4.461,1,0.0805,0.000821,0.000580,0.3580,0.216,78.056,audio_features,3xgK660fsZH7ZDcOMfIdfB,https://api.spotify.com/v1/tracks/3xgK660fsZH7...,https://api.spotify.com/v1/audio-analysis/3xgK...,189707,4
4,spotify:track:7o2CTH4ctstm8TNelqjb51,Sweet Child O' Mine,Guns N' Roses,78,"[glam metal, hard rock, rock]",Appetite For Destruction,19,0.454,0.910,6,-7.766,1,0.0448,0.086600,0.099600,0.1160,0.629,125.116,audio_features,7o2CTH4ctstm8TNelqjb51,https://api.spotify.com/v1/tracks/7o2CTH4ctstm...,https://api.spotify.com/v1/audio-analysis/7o2C...,354520,4


In [46]:
df.describe()

,artist_pop,track_pop,danceability,energy,keys,loudness,modes,speechiness,acousticness,instrumentalness,liveness,valences,tempos,durations_ms,time_signatures
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0
mean,54.140000,42.130000,0.600240,0.785560,4.850000,-5.689860,0.760000,0.062851,0.107456,0.009270,0.199204,0.546240,119.673770,212675.010000,4.0
std,16.355285,22.374975,0.114338,0.120654,3.517273,2.178224,0.429235,0.051054,0.142812,0.039135,0.170114,0.192516,27.432697,39241.375504,0.0
min,16.000000,0.000000,0.212000,0.318000,0.000000,-18.064000,0.000000,0.027300,0.000014,0.000000,0.033300,0.150000,74.110000,126521.000000,4.0
25%,43.000000,29.500000,0.532500,0.714750,2.000000,-6.437250,1.000000,0.035575,0.007848,0.000000,0.087900,0.415500,101.511250,189337.000000,4.0
50%,58.000000,44.500000,0.613000,0.813000,5.000000,-5.357500,1.000000,0.045100,0.050350,0.000002,0.124500,0.562000,117.151000,207724.000000,4.0
75%,67.000000,57.500000,0.683000,0.865250,7.250000,-4.439250,1.000000,0.071850,0.166750,0.000164,0.275000,0.680500,126.048750,228229.500000,4.0
max,81.000000,85.000000,0.821000,0.970000,11.000000,-2.333000,1.000000,0.408000,0.622000,0.283000,0.906000,0.980000,207.508000,354520.000000,4.0


## Automate Playlist Track Data Extraction with Offsetting

In [85]:
def construct_storage():
    store_store = {
        'uris': [],
        'names': [],
        'artist_names': [],
        'artist_pop': [],
        'artist_genres': [],
        'albums': [],
        'track_pop': [],
        'danceability': [],
        'energy': [],
        'keys': [],
        'loudness': [],
        'modes': [],
        'speechiness': [],
        'acousticness': [],
        'instrumentalness': [],
        'liveness': [],
        'valences': [],
        'tempos': [],
        'types': [],
        'ids': [],
        'track_hrefs': [],
        'analysis_urls': [],
        'durations_ms': [],
        'time_signatures': []
    }
    return store_store

def process_items(store, items):
    for item in items:
        track_uri = item['track']['uri']
        store['uris'].append(track_uri)  # Retrieve track uri
        store['names'].append(item['track']['name'])  # Retrieve track name
    
        artist_uri = item['track']['artists'][0]['uri']  # Find artist uri
        artist_info = sp.artist(artist_uri)  # Extract information on artist
    
        store['artist_names'].append(item['track']['artists'][0]['name'])  # Access artist name
        store['artist_pop'].append(artist_info['popularity'])  # Access 
        store['artist_genres'].append(artist_info['genres'])  # Access artist genres
    
        store['albums'].append(item['track']['album']['name'])  # Access album names
        store['track_pop'].append(item['track']['popularity'])  # Access track popularity
    
        track_features = sp.audio_features(track_uri)[0]  # Access track features
        store['danceability'].append(track_features['danceability'])
        store['energy'].append(track_features['energy'])
        store['keys'].append(track_features['key'])
        store['loudness'].append(track_features['loudness'])
        store['modes'].append(track_features['mode'])
        store['speechiness'].append(track_features['speechiness'])
        store['acousticness'].append(track_features['acousticness'])
        store['instrumentalness'].append(track_features['instrumentalness'])
        store['liveness'].append(track_features['liveness'])
        store['valences'].append(track_features['valence'])
        store['tempos'].append(track_features['tempo'])
        store['types'].append(track_features['type'])
        store['ids'].append(track_features['id'])
        store['track_hrefs'].append(track_features['track_href'])
        store['analysis_urls'].append(track_features['analysis_url'])
        store['durations_ms'].append(track_features['duration_ms'])
        store['time_signatures'].append(track_features['time_signature'])
    return store

def retrieve_batch_info(playlist, store):
    items = playlist['items']
    store = process_items(store, items)
    return store
    

In [121]:
def extract_tracks(spotify, playlist_uri, store):
    offset = 0
    limit = 100
    playlist = sp.playlist_tracks(playlist_uri, limit=100, offset=offset)  # Retrieve the initial batch of songs
    time.sleep(35)  # Allow for time imbetween API calls to respect rate limiting
    total_songs = playlist['total']  # Extract the total number of songs
    print(f"Total songs: {total_songs}")

    while offset < total_songs:
        time.sleep(10)
        playlist = sp.playlist_tracks(playlist_uri, limit=100, offset=offset)  # Retrieve batch of songs in playlist
        retrieve_batch_info(playlist, store)  # Retrieve batch information
        print(f"Current offset: {offset}")
        offset = offset + limit  # Update offset
    return store

In [122]:
store = construct_storage()  # Construct info storage
store = extract_tracks(sp, playlist_uri, store)

Total songs: 301


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=3V9cM3nCH2G66afoDi0snu:
 Max Retries, reason: too many 429 error responses

In [ ]:
print(store)

In [ ]:
len(store['uris'])

## Access Top-performing Playlists per Country

Access current/ top performing playlists in order to extract playlist uri's for automated track data extraction. 

In [52]:
playlists = sp.featured_playlists(country='GB', limit=20)

In [53]:
playlist_items = playlists['playlists']['items']

In [54]:
for item in playlist_items[0:10]:
    print(item['uri'])
    print(item['name'])

spotify:playlist:37i9dQZF1DX1OY2Lp0bIPp
Monday Motivation
spotify:playlist:37i9dQZF1DXcDoDDetPsEg
Who We Be
spotify:playlist:37i9dQZF1DXa71eg5j9dKZ
Altar
spotify:playlist:37i9dQZF1DWY4lFlS4Pnso
Hot Hits UK
spotify:playlist:37i9dQZF1DX9Z3vMB2b8im
The Most Beautiful Songs in the World
spotify:playlist:37i9dQZF1DWXU4C3nPzApC
Very Nearly Nashville
spotify:playlist:37i9dQZF1DWSQScAbo5nGF
Indie Roadtrip
spotify:playlist:37i9dQZF1DX7364T8tu1TH
The Dance List
spotify:playlist:37i9dQZF1DXbHcQpOiXk1D
Jazz UK
spotify:playlist:37i9dQZF1DXdeMORbC1XNa
ACS


In [71]:
def find_top_playlists(country):
    uris = []
    names = []
    playlists = sp.featured_playlists(country=country, limit=20)
    playlist_items = playlists['playlists']['items']
    for item in playlist_items:
        uris.append(item['uri'].split(':')[-1])
        names.append(item['name'])
    return uris, names

In [61]:
top_playlists, names = find_top_playlists('GB')
print(top_playlists)
print(names)

['37i9dQZF1DX1OY2Lp0bIPp', '37i9dQZF1DXcDoDDetPsEg', '37i9dQZF1DXa71eg5j9dKZ', '37i9dQZF1DWY4lFlS4Pnso', '37i9dQZF1DX9Z3vMB2b8im', '37i9dQZF1DWXU4C3nPzApC', '37i9dQZF1DWSQScAbo5nGF', '37i9dQZF1DX7364T8tu1TH', '37i9dQZF1DXbHcQpOiXk1D', '37i9dQZF1DXdeMORbC1XNa']
['Monday Motivation', 'Who We Be', 'Altar', 'Hot Hits UK', 'The Most Beautiful Songs in the World', 'Very Nearly Nashville', 'Indie Roadtrip', 'The Dance List', 'Jazz UK', 'ACS']


Confirm extracted uri's work with the created track extraction function

In [57]:
tracks = extract_tracks(sp, top_playlists[0])

Total songs: 100
Current offset: 0


In [58]:
print(tracks)

{'uris': ['spotify:track:6UelLqGlWMcVH1E5c4H7lY', 'spotify:track:0nrRP2bk19rLc0orkWPQk2', 'spotify:track:7JJmb5XwzOO8jgpou264Ml', 'spotify:track:003vvx7Niy0yvhvHt4a68B', 'spotify:track:6RUKPb4LETWmmr3iAEQktW', 'spotify:track:6K4t31amVTZDgR3sKmwUJJ', 'spotify:track:6DCZcSspjsKoFjzjrWoCdn', 'spotify:track:2374M0fQpWi3dLnB54qaLX', 'spotify:track:7KXjTSCq5nL1LoYtL7XAwS', 'spotify:track:4bHsxqR3GMrXTxEPLuK5ue', 'spotify:track:4kbj5MwxO1bq9wjT5g9HaA', 'spotify:track:2JzZzZUQj3Qff7wapcbKjc', 'spotify:track:3ZOEytgrvLwQaqXreDs2Jx', 'spotify:track:3GCdLUSnKSMJhs4Tj6CV3s', 'spotify:track:2V65y3PX4DkRhy1djlxd9p', 'spotify:track:2tUBqZG2AbRi7Q0BIrVrEj', 'spotify:track:1auxYwYrFRqZP7t3s7w4um', 'spotify:track:1KGi9sZVMeszgZOWivFpxs', 'spotify:track:6FE2iI43OZnszFLuLtvvmg', 'spotify:track:3bidbhpOYeV4knp8AIu8Xn', 'spotify:track:1FTSo4v6BOZH9QxKc3MbVM', 'spotify:track:5r5cp9IpziiIsR6b93vcnQ', 'spotify:track:0ikz6tENMONtK6qGkOrU3c', 'spotify:track:5Hroj5K7vLpIG4FNCRIjbP', 'spotify:track:7Ie9W94M7OjPoZV

## Data Collection

Specify the countries from which to extract the top 20 playlists from: 
- South Africa (**ZA**)
- Australia (**AU**)
- United Kingdom (**GB**)
- United States (**UK**)
- Canada (**CA**)
- Jamaica (**JM**)
- Malta (**MT**)
- Netherlands (**NL**)
- France (**FR**)
- Germany (**DE**)
- Ghana (**GH**)

In [88]:
def add_playlist_tracking(name, tracks):
    tracks['playlist_name'] = [name] * len(tracks['uris'])
    return tracks


def record_playlists(top_playlists, names, playlist_store, name_store):
    playlist_store.extend(top_playlists)
    name_store.extend(names)

In [111]:
countries = ['AU', 'GB', 'US', 'CA', 'JM', 'MT', 'NL', 'FR', 'DE', 'GH', 'ZA']

In [120]:
playlist_store = []
name_store = []  # Construct playlist info storage
store = construct_storage()  # Construct track info storage

for country in countries: 
    print(f'Country: {country}') 
    top_playlists, names = find_top_playlists(country)

    for playlist, name in zip(top_playlists, names):
        print(f'Playlist name: {name}')
        tracks = extract_tracks(sp, playlist, store)
        tracks = add_playlist_tracking(name, tracks)
        time.sleep(45)

    record_playlists(top_playlists, names, playlist_store, name_store)
    print('-----------------------------------------------------------------------------')

Country: AU
Playlist name: Get Popped!
Total songs: 87


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=2wWX4sZGZNc7lA594ZOhP7:
 Max Retries, reason: too many 429 error responses

In [ ]:
print(playlist_store)
print(name_store)
print("Total Playlists: ", len(name_store))

In [ ]:
print(store)

In [ ]:
print(len(store['uris']))